# Assignment 1: Dynamic Programming

---

## Task 1) Edit Distances

Implement the [Hamming](https://en.wikipedia.org/wiki/Hamming_distance) and [Levenshtein](https://en.wikipedia.org/wiki/Levenshtein_distance) (edit) distances and compute them for the for the following word pairs.
It may help to compute them by hand first.

<img src = "./assets/97090.jpg" width="33%" /> <img src = "./assets/97222.jpg" width="33%" /> <img src = "./assets/97669.jpg" width="33%" />

Aside from computing the distance (which is a scalar), do the backtrace and compute the edit transcript (and thus alignment).

---

In [4]:
WORD_PAIRS = [
    ("GCGTATGAGGCTAACGC", "GCTATGCGGCTATACGC"),
    ("kühler schrank", "schüler krank"),
    ("the longest", "longest day"),
    ("nicht ausgeloggt", "licht ausgenockt"),
    ("gurken schaben", "schurkengaben")
]

In [5]:
def hamming(s1: str, s2: str) -> int:
    """
    Compute the hamming distance between two strings.

    Arguments:
    s1: First string of word pair.
    s2: Second string of word pair.

    Returns the distance as int.
    """
    # Hint: Think about how you can deal with unequal word lengths.
    
    ### YOUR CODE HERE
    distance = 0
    #assert len(s1) == len(s2), "Strings must have equal length"
    for i in range(max(len(s1), len(s2))):

        if i>=len(s1) or i >= len(s2) or  s1[i] != s2[i]:
            distance += 1
    return distance


    ### END YOUR CODE

In [6]:
for wordpair in WORD_PAIRS:
    dist = hamming(s1=wordpair[0], s2=wordpair[1])
    print("hamming('{}', '{}') = {}".format(
        wordpair[0], wordpair[1], dist
    ))

### EXPECTED
# hamming('GCGTATGAGGCTAACGC', 'GCTATGCGGCTATACGC') = 10
# hamming('kühler schrank', 'schüler krank') = 13
# hamming('the longest', 'longest day') = 11
# hamming('nicht ausgeloggt', 'licht ausgenockt') = 4
# hamming('gurken schaben', 'schurkengaben') = 14

hamming('GCGTATGAGGCTAACGC', 'GCTATGCGGCTATACGC') = 10
hamming('kühler schrank', 'schüler krank') = 13
hamming('the longest', 'longest day') = 11
hamming('nicht ausgeloggt', 'licht ausgenockt') = 4
hamming('gurken schaben', 'schurkengaben') = 14


In [7]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [8]:
import numpy as np
def levenshtein(s1: str, s2: str, cost={'m': 0, 's': 1, 'i': 1, 'd': 1}) -> (int, str):
    """
    Compute the levenshtein (edit) distance between two strings.

    Arguments:
    s1: First string of word pair.
    s2: Second string of word pair.

    Returns the distance as int and edit transcript as string.
    """
    # Hint: The probably most intuitive approach is bottom-up.
    
    ### YOUR CODE HERE
    #print("s1: ", s1, "s2: ", s2)
    costs = np.zeros((len(s1)+1, len(s2)+1), dtype=int) # kosten matrix

    #erste Zeile und spalte 
    costs[0, 1:] = range(1, len(s2) + 1)
    costs[1:, 0] = range(1, len(s1) + 1)

    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            if s1[i-1] == s2[j-1]:
                delta = cost['m']
            else:
                delta = cost['s']
            costs[i, j] = min(costs[i-1, j] + cost['d'], costs[i, j-1] + cost['i'], costs[i-1, j-1] + delta)

    transcript = ""
    #backtrack
    i = len(s1)
    j = len(s2)
    while i > 0 and j > 0:
        if s1[i-1] == s2[j-1]:
            transcript += "m"
            i -= 1
            j -= 1
        elif costs[i, j] == costs[i-1, j] + cost['d']:
            transcript += "d"
            i -= 1
        elif costs[i, j] == costs[i, j-1] + cost['i']:
            transcript += "i"
            j -= 1
        else:
            transcript += "s"
            i -= 1
            j -= 1
    #print(costs)
    return costs[-1, -1], transcript[::-1]
    ### END YOUR CODE

In [9]:
for wordpair in WORD_PAIRS:
    dist, transcript = levenshtein(s1=wordpair[0], s2=wordpair[1])
    print("levenshtein('{}', '{}') = {} ({})".format(
        wordpair[0], wordpair[1], dist, transcript
    ))

### EXPECTED
# levenshtein('GCGTATGAGGCTAACGC', 'GCTATGCGGCTATACGC') = 3 (mmdmmmmsmmmmmimmmm)
# levenshtein('kühler schrank', 'schüler krank') = 6 (ssmimmmmsddmmmm)
# levenshtein('the longest', 'longest day') = 8 (ddddmmmmmmmiiii)
# levenshtein('nicht ausgeloggt', 'licht ausgenockt') = 4 (smmmmmmmmmmsmssm)
# levenshtein('gurken schaben', 'schurkengaben') = 7 (siimmmmmsdddmmmm)

levenshtein('GCGTATGAGGCTAACGC', 'GCTATGCGGCTATACGC') = 3 (mmdmmmmsmmmmmimmmm)
levenshtein('kühler schrank', 'schüler krank') = 6 (ssmimmmmsddmmmm)
levenshtein('the longest', 'longest day') = 8 (mmmmmmmiiii)
levenshtein('nicht ausgeloggt', 'licht ausgenockt') = 4 (smmmmmmmmmmsmssm)
levenshtein('gurken schaben', 'schurkengaben') = 7 (siimmmmmsdddmmmm)


---

## Task 2) Basic Spelling Correction using Edit Distance

For spelling correction, we will use prior knowledge, to put _some_ learning into our system.

The underlying idea is the _Noisy Channel Model_, that is: The user _intends_ to write a word `w`, but through some noise in the process, happens to type the word `x`.

The correct word $\hat{w}$ is that word, that is a valid candidate and has the highest probability:

$$
\begin{eqnarray}
\DeclareMathOperator*{\argmax}{argmax}
\hat{w} & = & \argmax_{w \in V} P(w | x) \\
        & = & \argmax_{w \in V} \frac{P(x|w) P(w)}{P(x)} \\
        & = & \argmax_{w \in V} P(x|w) P(w)
\end{eqnarray}
$$

1. The candidates $V$ can be obtained from a _vocabulary_.
2. The probability $P(w)$ of a word $w$ can be _learned (counted) from data_.
3. The probability $P(x|w)$ is more complicated... It could be learned from data, but we could also use a _heuristic_ that relates to the edit distance, e.g. rank by distance.

You may use [Peter Norvig's count_1w.txt](http://norvig.com/ngrams/) file, [local mirror](res/count_1w.tar.bz2).
Note that it may help to restrict to the first 10k words to get started.

---

In [10]:
EXAMPLES = [
    "pirates",    # in-voc
    "pirutes",    # pirates?
    "continoisly",  # continuosly?
]

In [11]:
%pip install pandas
import pandas as pd


Note: you may need to restart the kernel to use updated packages.


In [18]:
### TODO: Prepare the l

### YOUR CODE HERE
words= pd.read_csv("./data/count_1w.txt", sep="\t", header=None, names=["word","count"],dtype={"word":str,"count":int})
words= words.head(10000)


### END YOUR CODE

In [20]:
words

,word,count
0,the,23135851162
1,of,13151942776
2,and,12997637966
3,to,12136980858
4,a,9081174698
...,...,...
9995,varieties,5057493
9996,arbor,5057261
9997,mediawiki,5056973
9998,configurations,5056310


In [54]:
def suggest(w: str, dist_fn, max_cand=5) -> list:
    """
    Compute suggestions for spelling correction using edit distance.
    
    Arguments:
    w: Word in question.
    dist_fn: Distance function to use (e.g. levenshtein).
    max_cand: Maximum number of suggestions.

    Returns a list of tuples (word, dist, score) sorted by score and distance.
    """
    mywords= pd.DataFrame(words)
    ### YOUR CODE HERE
    mywords["sim"]= 0.0
    for i in range(len(mywords)):
        mywords.at[i, "sim"], _ = dist_fn(w, str(mywords.at[i, "word"]))
    
        #mywords.at[i, "score"]= float(mywords.at[i, "count"]) / float(mywords.at[i, "sim"]+1)
    mywords = mywords.sort_values(by=[ "sim"], ascending= True)
    return  mywords.head(max_cand)

    ### END YOUR CODE
    

In [22]:
# How does your suggest function behave with levenshtein distance?

for word in EXAMPLES:
    suggestions = suggest(w=word, dist_fn=levenshtein, max_cand=3)
    print("{} {}".format(word, suggestions))

### EXPECTED
### Notice: your scores may vary!
# pirates [('pirates', 0, -11.408058827802126)]
# pirutes [('pirates', 1, -11.408058827802126), ('minutes', 2, -8.717825438953103), ('viruses', 2, -11.111468702571859)]
# continoisly [('continously', 1, -15.735337826575178), ('continuously', 2, -11.560071979871001), ('continuosly', 2, -17.009283000138204)]

pirates           word    count  sim         score
8415   pirates  6531487  0.0  6.531487e+06
7878  emirates  7182004  2.0  2.394001e+06
8351   phrases  6596622  2.0  2.198874e+06
pirutes          word     count  sim         score
8415  pirates   6531487  1.0  3.265744e+06
6816  viruses   8786573  2.0  2.928858e+06
831   minutes  96242209  2.0  3.208074e+07
continoisly               word    count  sim         score
9348  continuously  5610397  2.0  1.870132e+06
8861    continuity  6045562  3.0  1.511390e+06
9705   continually  5278721  3.0  1.319680e+06


### Food for Thought

- How would you modify the implementation so that it works fast for large lexica (eg. the full file above)?
- How would you modify the implementation so that it works "while typing" instead of at the end of a word?
- How do you handle unknown/new words?

---

## Task 3) Needleman-Wunsch: Keyboard-aware Auto-Correct

In the parts 1 and 2 above, we applied uniform cost to all substitutions.
This does not really make sense if you look at a keyboard: the QWERTY layout will favor certain substitutions (eg. _a_ and _s_), while others are fairly unlikely (eg. _a_ and _k_).

Implement the [Needleman-Wunsch algorithm](https://en.wikipedia.org/wiki/Needleman–Wunsch_algorithm) which is very similar to the [Levenshtein distance](https://en.wikipedia.org/wiki/Levenshtein_distance), however it doesn't _minimize the cost_ but _maximizes the similarity_.
For a good measure of similarity, implement a metric that computes a meaningful weight for a given character substitution.

---

In [25]:
import math

In [1]:

# QWERTY key positions (row, col)
keyboard = {
    'q': (0, 0), 'w': (0, 1), 'e': (0, 2), 'r': (0, 3), 't': (0, 4), 'y': (0, 5), 'u': (0, 6), 'i': (0, 7), 'o': (0, 8), 'p': (0, 9),
    'a': (1, 0), 's': (1, 1), 'd': (1, 2), 'f': (1, 3), 'g': (1, 4), 'h': (1, 5), 'j': (1, 6), 'k': (1, 7), 'l': (1, 8),
    'z': (2, 0), 'x': (2, 1), 'c': (2, 2), 'v': (2, 3), 'b': (2, 4),  'n': (2, 5), 'm': (2, 6)
}

def keyboard_sim(c1: str, c2: str) -> float:
    """abstand zwischen zwei chats im qwerty layout"""
    c1, c2 = c1.lower(), c2.lower()
    if c1 not in keyboard or c2 not in keyboard:
        return 0.0
    pos1, pos2 = keyboard[c1], keyboard[c2]
    d = math.sqrt((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)
    return (1 - (d / 9.5))*20
# higher is closer 

def nw(s1: str, s2: str, d: float, sim_fn) -> float:
    """
    Apply needleman-wunsch algorithm.
    
    Arguments:
    s1: First string of word pair.
    s2: Second string of word pair.
    d: Gap penalty score.
    sim_fn: Similarity function to use.

    Returns the score as float.
    """
    ### YOUR CODE HERE
    D = np.zeros((len(s1) + 1, len(s2) + 1), dtype=int)
# for the empty word, costs match the length of the other string
    D[0, 1:] = range(1, len(s2) + 1); D[0, 1:] *= d
    D[1:, 0] = range(1, len(s1) + 1); D[1:, 0] *= d
    for i in range(1, len(s1)):
        for j in range(1, len(s2)):
            cs = D[i-1, j-1] + sim_fn(s1[i], s2[j])
            cd = D[i-1, j] + d
            ci = D[i, j-1] + d
            D[i,j] = max(cs, cd, ci)
            #print(D)

    return D[len(s1)-1][len(s2)-1]    
    ### END YOUR CODE


def nw_based_dist(s1: str, s2: str) -> (int, str):
    """
    Compute the needleman-wunsch distance between two strings.
    
    Arguments:
    s1d: First string of word pair.
    s2: Second string of word pair.
    
    Returns the distance as int and <unsupported> string.
    """
    ### YOUR CODE HERE
    
    return nw(s1, s2, -1, keyboard_sim), "<unsupported>"
    # Hint: return dist, "<unsupported>"
    
    ### END YOUR CODE

In [52]:
nw("halo", "helo",-1, keyboard_sim)


np.int64(2)

In [2]:
# How does your suggest function behave with nw and a keyboard-aware similarity?
# irgendwie kommt da bullshit raus warum auch immer...
for word in EXAMPLES:
    suggestions = suggest(w=word, dist_fn=nw_based_dist, max_cand=3)
    print("{} {}".format(word, suggestions))

NameError: name 'EXAMPLES' is not defined

### Food for Thought

- What could be better heuristics for similarity resp. cost of substitution than the one above?
- What about capitalization, punctiation and special characters?
- What about swipe-to-type?